In [135]:
import pandas as pd
from pandas_gbq import read_gbq
import logging
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
from typing import List
import argparse

## Config

In [41]:
SQL_PATH = '../sql/'
OUTPUT_PATH = '/mnt/windows_mount/FILE/'

projectid = 'ts-official-website'
private_key = "../key/TS-Official-Website-d438f1c2b14d.json"

configuration = {
   'query': {
     "useQueryCache": True
   }
}

### Logging

In [4]:
pass

### SQL Loader

In [11]:
sql_path = Path(SQL_PATH)
list(sql_path.resolve().glob('*.sql'))

[PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_EVENT_INFO.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_CUSTOM_METRICS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_CUSTOM_DIMENSIONS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_PAGE.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_HITS_SOCIAL.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_TRAFFIC_SOURCE.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA_MAIN_TOTALS.sql'),
 PosixPath('/home/ts1050391/Dropbox/Taishin/taishin_works/ga_data_fetch/sql/GA

In [45]:
sql_files = sql_path.resolve().glob('*.sql')

sqls = []
for _, file_path in enumerate(sql_files):
    with open(file_path, 'r', encoding='utf8') as file:
        try:
            sqls.append(file.read())
        except Exception as e:
            print(e)

In [7]:
len(sqls)

18

### get_bq_result

In [58]:
sql = sqls[2].format(start_date='20180314', end_date='20180314')
print(sql)

--GA_MAIN_HITS--
--官網--

SELECT  TO_HEX(SHA256(Concat(cast(FullvisitorID as string),"_",cast(VISITID as string)))) as GA_MAIN_NO ,
        "Official_Web" as DATASOURCE,
        FullvisitorID as FULL_VISITOR_ID,
        VISITID as VISIT_ID,
        HITS.hitnumber as HIT_NUMBER,
        HITS.hour as HOUR,
        HITS.isEntrance as IS_ENTRANCE,
        HITS.isEXIT as IS_EXIT,
        HITS.isInteraction AS IS_INTERACTION,
        HITS.minute as MINUTE,
        HITS.TIME as TIME,
        HITS.REFerer as REFERER,
        hits.type as TYPE
FROM `ts-official-website.118166279.ga_sessions_*`,
        UNNEST(hits) as hits
WHERE _TABLE_SUFFIX BETWEEN '20180314' and '20180314'


In [59]:
%%time

df = read_gbq(
    sql,
    private_key=private_key,
    project_id=projectid,
    configuration=configuration,
    dialect='standard'
)

In [60]:
df.head(10)

,GA_MAIN_NO,DATASOURCE,FULL_VISITOR_ID,VISIT_ID,HIT_NUMBER,HOUR,IS_ENTRANCE,IS_EXIT,IS_INTERACTION,MINUTE,TIME,REFERER,TYPE
0,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,1,9,True,None,True,17,0,None,PAGE
1,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,2,9,None,None,True,17,4344,None,PAGE
2,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,3,9,None,None,True,17,10216,None,PAGE
3,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,4,9,None,None,True,17,19624,None,PAGE
4,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,5,9,None,None,True,17,23008,None,PAGE
5,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,6,9,None,None,True,18,37856,None,PAGE
6,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,7,9,None,None,True,18,43208,None,PAGE
7,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,8,9,None,None,True,18,48992,None,PAGE
8,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,9,9,None,None,True,18,53344,None,PAGE
9,ca62116aeae6c4f6171239349ef56a69710a34157e69a5...,Official_Web,6336352424218817447,1520990255,10,9,None,None,True,18,58360,None,PAGE


## Refactor

In [148]:
def main(argv: List[str]) -> int:
    
    # Arg parser
    args = parse_args(argv[1:])
    if args.conf_file:
        config = ConfigParser.SafeConfigParser()
        config.read([args.conf_file])
        defaults = dict(config.items("Defaults"))
    
    start_date = args.start_date  # '20180101'
    end_date = args.end_date  # '20180101'
    
    # config
    if args.config_file:
        SQL_PATH = '../sql/'
        PROJECT_ID = 'ts-official-website'
        OUTPUT = '/mnt/windows_mount/FILE/'
        KEY = "../key/TS-Official-Website-d438f1c2b14d.json"
    else:
        raise ValueError('Config file required.')

    # Run
    tic = datetime.now()
    
    sqls = load_sql(SQL_PATH)
    
    res = {}
    for filename, sql in sqls.items():
        res.update(save_bq_csv(filename, sql, PROJECT_ID, start_date, end_date, OUTPUT))
    
    res = pd.DataFrame.from_dict({'key': list(res.keys()), 'val': list(res.values())})
    
    print(res)
    print('-------------------------------------------'
          f'\nDone. (Time elapsed: {datetime.now() - tic:.2f})')
    
    return 0

In [ ]:
def parse_args(argv: List[str]) -> argparse.Namespace:
    
    YESTERDAY = (datetime.today().date() + timedelta(-1)).strftime('%Y%m%d')
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', dest='config_file', required=True)
    parser.add_argument('-s', 'start-date', dest='start_date', default=YESTERDAY)
    parser.add_argument('-e', '--end-date', dest='end_date', default=YESTERDAY)

In [126]:
def load_sql(sql_path):
    '''Read sql to dict'''
    sql_files = Path(sql_path).resolve().glob('*.sql')

    sqls = {}
    for _, file_path in enumerate(sql_files):
        with open(file_path, 'r', encoding='utf8') as file:
            try:
                sql_str = file.read()
                sqls[file_path.stem] = sql_str
            except Exception as e:
                print(e)
    
    return sqls

In [127]:
def save_bq_csv(filename, sql, projectid, start_date, end_date, output):
    
    output = Path(output)
    if not output.is_dir():
        raise ValueError(f'`output` must be a folder path ({output})')
    
    res = {}
    for data_date in _dates_between(start_date, end_date):
        out_file = f'{filename}_{data_date}.csv'
        out_path = output / out_file
        print(f'Saving file: {str(out_path)}')
        res[out_file] = save_bq_csv_oneday(sql, projectid, data_date, output=str(out_path))
    
    return res

In [128]:
def save_bq_csv_oneday(sql, projectid, data_date, output, dialect='standard'):
    
    configuration = {
       'query': {
         "useQueryCache": True
       }
    }

    try:
        df = read_gbq(
            sql.format(start_date=data_date, end_date=data_date),
            private_key=KEY,
            project_id=projectid,
            configuration=configuration,
            dialect=dialect
        )

        df.to_csv(output, index=False)
        return df.shape[0]
    except Exception as e:
        print(e)
        return np.NaN

In [129]:
def _dates_between(start_date, end_date):
    
    def _date_range(date1, date2):
        for n in range(int((date2 - date1).days) + 1):
            yield date1 + timedelta(n)
    
    start_date = datetime.strptime(start_date, '%Y%m%d').date()
    end_date = datetime.strptime(end_date, '%Y%m%d').date()
    
    for dt in _date_range(start_date, end_date):
        yield dt.strftime("%Y%m%d")

### Run

In [143]:
# config
SQL_PATH = '../sql/'
OUTPUT = '/mnt/windows_mount/FILE/'
START_DATE = '20180101'
END_DATE = '20180101'

# BigQuery
projectid = 'ts-official-website'
private_key = "../key/TS-Official-Website-d438f1c2b14d.json"

In [131]:
sqls = load_sql(SQL_PATH)

In [132]:
print(list(sqls.items())[17])

('GA_MAIN_HITS_CUSTOM_VARIABLES', "--GA_MAIN_HITS_CUSTOM_VARIABLES--\n--行銷平台--\n\nSELECT TO_HEX(SHA256(Concat(cast(FullvisitorID as string),'_',cast(VISITID as string)))) AS GA_MAIN_NO,\n\t   FullvisitorID as FULL_VISITOR_ID ,visitID as VISIT_ID ,h.hitNumber as HIT_NUMBER ,\n\t   hcv.index AS INDEX, \n\t   hcv.customVarName AS CUSTOM_VAR_NAME, \n\t   hcv.customVarValue AS CUSTOM_VAR_VALUE\nFROM `ts-official-website.164284491.ga_sessions_*`,\n\t   UNNEST(hits) AS h,\n\t   UNNEST(h.customVariables) AS hcv\nWHERE _TABLE_SUFFIX BETWEEN '{start_date}' and '{end_date}'")


In [88]:
list(_dates_between('20180130', '20180205'))

['20180130',
 '20180131',
 '20180201',
 '20180202',
 '20180203',
 '20180204',
 '20180205']

In [133]:
if __name__ == '__main__':
    res = main(['run.py', ''])

'__main__'